<img src="https://nlp.johnsnowlabs.com/assets/images/logo.png" width="180" height="50" style="float: left;">

# Global DEMO version 2.2.1

## Example for Named Entity Recognition with Entity Resolution Pipeline
A common NLP problem in biomedical aplications is to identify the presence of clinical entities in a given text. This clinical entities could be diseases, symptoms, drugs, results of clinical investigations or others.

In this example we will use Spark-NLP to identify all the entities present in a typical clinical note.

The clinical note (taken from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6553675/) is as follows:

<div style="border:2px solid #747474; background-color: #e3e3e3; margin: 5px; padding: 10px"> 
<p>A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.</p>

<p>Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) 10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia.</p>

<p>The patient was initially admitted for starvation ketosis, as she reported poor oral intake for three days prior to admission. However, serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again. The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within 24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She had close follow-up with endocrinology post discharge.</p>
</div>

We will use Spark-NLP capabilities to identify a list of medical problems, treatments and medical tests, and then try to assign a ICD-10 code to each element of this list.

### Step 1. Prepare the environment

#### Install OpenSource spark-nlp and pyspark pip packages
As a first step we import the required python dependences including some sparknlp components.

Be sure that you have the required python libraries (pyspark 2.4.3, spark-nlp 2.2.1) by running <code>pip list</code>. Check that the versions are correct.

If some of them is missing you can run:

<code>pip install --ignore-installed pyspark==2.4.3</code><br>
<code>pip install --ignore-installed spark-nlp==2.2.1</code><br>

The --ignore-installed parameter is to overwrite your previous pip package version if already installed.

<i>*If this cell fails means you have not propertly setup the required environment. Please check the pre-requisites guideline at http://www.johnsnowlabs.com</i>

In [1]:
import sys, os, time

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.util import *
from sparknlp.embeddings import *

from sparknlp.embeddings import EmbeddingsHelper
from sparknlp.pretrained import ResourceDownloader

from pyspark.ml import Pipeline, PipelineModel


#### Install Licensed Sparl-NLP package

We will use also some Spark-NLP enterprise functionalities contained in the spark-nlp-jsl package.

You can check that spark-nlp-jsl is installed by running <code>pip install</code>. Check that version installed is 2.2.1.

If it is not then you need to install it by using:

<code>pip install spark-nlp-jsl==2.2.1 --extra-index-url #### --ignore-installed</code>

The #### is a secret url, if you have not received it please contact us at info@johnsnowlabs.com.

<i>*If the next cell fails means your licensed enterprise version is not propertly installed so please check the pre-requisites guideline at http://www.johnsnowlabs.com/</i>

In [2]:
# If this fails, means pip module for enterprise has not been properly set up

from sparknlp_jsl.annotator import *

#### Setup credentials to private JohnSnowLabs models repository with AWS-CLI

Now is time to configure Spark-NLP in order to access private JohnSnowLabs models repository. This access is done via Amazon aws command line interface (AWSCLI).

Instructions about how to install awscli are available at: 

https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html

Make sure you configure your credentials with <code>aws configure</code> following the instructions at:

https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html

Please substitute the ACCESS_KEY and SECRET_KEY with the credentials you have recived. If you need your credentials contact us at info@johnsnowlabs.com

#### Start Spark session

The following will initialize the spark session in case you have run the jupyter notebook directly. If you have started the notebook using pyspark this cell is just ignored.

Initializing the spark session takes some seconds (usually less than 1 minute) as the jar from the server needs to be loaded.

We will be using version 2.2.1 of Spark NLP Open Source and 2.2.1 of Spark NLP Enterprise Edition.

The #### in <code>.config("spark.jars", "####")</code> is a secret code, if you have not received it please contact us at info@johnsnowlabs.com.

In [3]:
# This cell will be ignored if jupyter started using pyspark
# This cell could take a while to run so be patient.
# We will 

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Global DEMO - Spark NLP Enterprise 2.2.1") \
    .master("local[*]") \
    .config("spark.driver.memory","4G") \
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:2.2.1") \
    .config("spark.jars", "####") \
    .getOrCreate()


### Step 2. Clinical NER Pipeline creation

In Spark-NLP annotating NLP happens through pipelines. Pipelines are made out of various Annotator steps. In our case the architecture of the Clinical Named Entity Recognition pipeline will be:

* DocumentAssembler (text -> document)
* SentenceDetector (document -> sentence)
* Tokenizer (sentence -> token)
* WordEmbeddingsModel ([sentence, token] -> embeddings)
* NerDLModel ([sentence, token, embeddings] -> ner)

So from a text we end having a list of Named Entities (Patient problems, Treatments and Tests).

#### Step 2.1 Initialize all the annotators required by the pipeline

The first 3 annotators of the pipeline are "DocumentAssembler", "SentenceDectector" and "Tokenizer":

In [4]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")



The fourth annotator in the pipeline is "WordEmbeddingsModel". We will download a pretrained model available from "clinical/models" named "embeddings_clinical".

When running this cell your are advised to be patient. 

First time you call this pretrained model it needs to be downloaded in your local.

The model size is about will download the embeddings_clinical corpus it takes a while.

The size is about 1.7Gb and will be saved typically in your home folder as

    ~HOMEFOLDER/cached_models/embeddings_clinical_en_2.0.2_2.4_1558454742956.zip

Next times you call it the model is loaded from your cached copy but even in that case it needs to be indexed each time so expect waiting up to 5 minutes (depending on your machine)

In [5]:
# WordEmbeddingsModel pretrained "embeddings_clinical" includes a model of 1.7Gb that needs to be downloaded

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


The fifth and final annotator in our NER pipeline is the pretrained "ner_clinical" NerDLModel avaliable from "clinical/models". It requires as input the "sentence", "token" and "embeddings" (clinical embeddings pretrained model) and will classify each token in four categories:
<ol>
    <li>PROBLEM: for patient problems</li>
    <li>TEST: for tests, labs, etc.</li>
    <li>TREATMENT: for treatments, medicines, etc.</li>
    <li>OTHER: for the rest of tokens.</li>
</ol>

In order to split those identified NER that are consecutive, the B prefix (as B-PROBLEM) will be used at the first token of each NER. The I prefix (as I-PROBLEM) will be used for the rest of tokens inside the NER.

In [6]:
# Named Entity Recognition for clinical concepts.

clinical_ner = NerDLModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")



ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[OK!]


#### Step 2.2 Define the NER pipeline

Now we will define the actual pipeline that puts together the annotators we have created.

In [7]:
# Build up the pipeline

pipeline_ner = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner
  ])

### Step 3 Create a SparkDataFrame with the content

Now we will create a sample Spark dataframe with our clinical note example.

In this example we are working over a unique clinical note. In production environments a table with several of those clinical notes could be distributed in a cluster and be run in large scale systems.

In [8]:
# reference https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6553675/

clinical_note = (
    'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years '
    'prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior '
    'episode of HTG-induced pancreatitis three years prior to presentation, associated '
    'with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, '
    'presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. '
    'Two weeks prior to presentation, she was treated with a five-day course of amoxicillin '
    'for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin '
    'for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months '
    'at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; '
    'significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent '
    'laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, '
    'creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) '
    '10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed '
    'as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for '
    'starvation ketosis, as she reported poor oral intake for three days prior to admission. However, '
    'serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap '
    'was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and '
    'lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - '
    'the original sample was centrifuged and the chylomicron layer removed prior to analysis due to '
    'interference from turbidity caused by lipemia again. The patient was treated with an insulin drip '
    'for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within '
    '24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting '
    'of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on '
    '40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg '
    'two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She '
    'had close follow-up with endocrinology post discharge.'
)

data_ner = spark.createDataFrame([[clinical_note]]).toDF("text")



In [9]:
data_ner.show()

+--------------------+
|                text|
+--------------------+
|A 28-year-old fem...|
+--------------------+



### Step 4 Create a model fiting the NER pipeline with the clinical note.

Now we can use the pipeline and the clinical note to generate a model.

In [10]:
# We convert the pipeline into a model fitting our clinical note (data).
model_ner = pipeline_ner.fit(data_ner)


### Step 5 Transform/annotate the clinical note using the model.

In order to process the data with the new created model we have two options.

The first one would be to use the model to transform our clinical note by the command:

<code>output = model_ner.transform(data_ner)</code>

That would save in a Spakr DataFrame (output) the resuls of running the model over the clinical note. 

However for small tests like this or for real-time request a LightPipelines is a simpler way of managing the data. It will return a dictionary (instead of a Spark DataFrame) with the results of the transformation

We will create a light_pipeline_ner using our model_ner and then will annotate the clinical_note using this light_pipeline.

In [11]:
light_pipeline = LightPipeline(model_ner)
light_data = light_pipeline.annotate(clinical_note)

Now we have a dictionaty (light_data_ner) that contains the results of running the NER pipeline over our clinical note.

It contains the original document:

In [12]:
light_data['document'][0][0:100]

'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to '

The 17 sentences:

In [13]:
print("Number of sentences: {}".format(len(light_data['sentence'])))
print("")
for i in range(5):
    print("Sentence {}: {}".format(i, light_data['sentence'][i][0:80]))

Number of sentences: 17

Sentence 0: A 28-year-old female with a history of gestational diabetes mellitus diagnosed e
Sentence 1: Two weeks prior to presentation, she was treated with a five-day course of amoxi
Sentence 2: She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and
Sentence 3: She had been on dapagliflozin for six months at the time of presentation.
Sentence 4: Physical examination on presentation was significant for dry oral mucosa;


And the 437 tokens with their assigned NER class.

In [14]:
print("Number of tokens: {}".format(len(light_data['token'])))
print("")
for i in range(25):
    print("Token {}: {} ({})".format(i, light_data['token'][i][0:20], light_data['ner'][i]))
print("...")

Number of tokens: 436

Token 0: A (O)
Token 1: 28-year-old (O)
Token 2: female (O)
Token 3: with (O)
Token 4: a (O)
Token 5: history (O)
Token 6: of (O)
Token 7: gestational (B-PROBLEM)
Token 8: diabetes (I-PROBLEM)
Token 9: mellitus (I-PROBLEM)
Token 10: diagnosed (O)
Token 11: eight (O)
Token 12: years (O)
Token 13: prior (O)
Token 14: to (O)
Token 15: presentation (O)
Token 16: and (O)
Token 17: subsequent (O)
Token 18: type (B-PROBLEM)
Token 19: two (I-PROBLEM)
Token 20: diabetes (I-PROBLEM)
Token 21: mellitus (I-PROBLEM)
Token 22: ( (O)
Token 23: T2DM (B-PROBLEM)
Token 24: ), (I-PROBLEM)
...


Lets apply some HTML formating to see the results of the pipeline in a nicer layout:

In [15]:
html_output = '<center><h2>Results of NER Annotation Pipeline</h2></center>'
html_output += '<div style="border:2px solid #747474; background-color: #e3e3e3; margin: 5px; padding: 10px">'
problem_flag = False
new_problem = []
problem_list = []
for index, this_token in enumerate(light_data['token']):
    
    if light_data['ner'][index] in ['B-PROBLEM','I-PROBLEM']:
        if problem_flag == False:
            new_problem = [this_token]
        else:
            new_problem.append(this_token)
        problem_flag = True
    else:
        if problem_flag == True:
            problem_list.append(new_problem)
            new_problem = []
        problem_flag = False
    
    if light_data['ner'][index]=='O':
        html_output+=this_token + " "
    elif light_data['ner'][index]=='B-PROBLEM':
        html_output+='<SPAN style="background-color: #ffffcc">' + this_token + " </SPAN>"
    elif light_data['ner'][index]=='I-PROBLEM':
        html_output+='<SPAN style="background-color: #ffffcc">' + this_token + " </SPAN>"
    elif light_data['ner'][index]=='B-TEST':
        html_output+='<SPAN style="background-color: pink">' + this_token + " </SPAN>"
    elif light_data['ner'][index]=='I-TEST':
        html_output+='<SPAN style="background-color: pink">' + this_token + " </SPAN>"
    elif light_data['ner'][index]=='B-TREATMENT':
        html_output+='<SPAN style="background-color:  #cce6ff">' + this_token + " </SPAN>"
    elif light_data['ner'][index]=='I-TREATMENT':
        html_output+='<SPAN style="background-color:  #cce6ff">' + this_token + " </SPAN>"
    

html_output += '</div>'
        
html_output += '<div>Color codes: <SPAN style="background-color: #ffffcc">Patient problem</SPAN>, '
html_output += '<SPAN style="background-color: pink">Test</SPAN>, '
html_output += '<SPAN style="background-color: #cce6ff">Treatment</SPAN>'

from IPython.core.display import display, HTML
display(HTML(html_output))

### Step 5 ICD10 coding Pipeline creation.

After running the NER Pipeline we have been able to extract a list of "Patient Problems" that is printed below:

In [16]:
for problem in problem_list:
    print(" ".join(problem))

gestational diabetes mellitus
type two diabetes mellitus
T2DM ),
HTG-induced pancreatitis
an acute hepatitis
obesity
a body mass index
polyuria
polydipsia
poor appetite
vomiting
a respiratory tract infection
T2DM
HTG
tenderness
guarding
rigidity
significant lipemia
starvation ketosis
poor oral intake
still elevated
interference from turbidity
lipemia
euDKA
HTG
a reduction in the anion gap
Her euDKA
her respiratory tract infection


We will now create a new pipeline that from each of these problems will try to assign an ICD10 base on the content, the wordembeddings and some pretrained models for ICD10 annotation.

The architecture of this new pipeline will be as follows:
* DocumentAssembler (text -> document)
* SentenceDetector (document -> sentence)
* Tokenizer (sentence -> token)
* WordEmbeddingsModel ([sentence, token] -> embeddings)
* NerDLModel ([sentence, token, embeddings] -> ner)
* NerConverter (["sentence, token, ner] -> ner_chunk
* ChunkTokenizer (ner_chunk -> ner_chunk_tokenized)
* ICD10CMEntityResolverModel ([ner_chunk_tokenized, embeddings] -> resolution)
* ICD10PCSEntityResolverModel ([ner_chunk_tokenized, embeddings] -> resolution)

So from a text we end having a list of Named Entities (ner_chunk) and their ICD10 codes (resolution)

Most of the annotators in this pipeline have been already created for the previous pipeline, but we need to create four additional annotators: NerConverter, ChunkTokenizer, EntityResolverModel for ICD10CM and EntityResolverModel for ICD10PCS.

In [17]:
# Named Entity Recognition concepts parser, transforms entities into CHUNKS (required for next step: assertion status)

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

# We have to tokenize the chunks, to fullfil EntityResolver requirements of TOKEN

chunk_tokenizer = ChunkTokenizer() \
  .setInputCols(["ner_chunk"]) \
  .setOutputCol("ner_chunk_tokenized")

# ICD resolution model

icd_resolution_cm = EntityResolverModel.pretrained("resolve_icd10cm_cl_em", "en", "clinical/models") \
  .setInputCols(["ner_chunk_tokenized", "embeddings"]) \
  .setOutputCol("resolution_cm") \
  .setThreshold(10)

#PCS Resolution
icd_resolution_pcs = EntityResolverModel.pretrained("resolve_icd10pcs_cl_em", "en", "clinical/models") \
  .setInputCols(["ner_chunk_tokenized", "embeddings"]) \
  .setOutputCol("resolution_pcs") \
  .setThreshold(10)

Now we define the new pipeline

In [18]:
# Build up the pipeline

pipeline_icd10 = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    chunk_tokenizer,
    icd_resolution_cm,
    icd_resolution_pcs
  ])

model_icd10 = pipeline_icd10.fit(data_ner)
light_pipeline_icd10 = LightPipeline(model_icd10)

For each of the patient problems identified in the clinical note, we can run the ICD10 resolution pipeline and in case we found a candidate ICD10 code we print the results.

In [19]:
# Now for the list of Patient Problem entities we will run the LightPipeline
problem_list_str = [" ".join(this_problem) for this_problem in problem_list]

print("====================================================================================")
print("ICD10 codes identified for the list of patients problems found in the clinical note.")
print("====================================================================================")

for problem in problem_list_str:
    this_r = light_pipeline_icd10.annotate(problem)
    if(len(this_r['resolution_cm'])>0):
        print("{} >>> ICD10CM: {}".format(this_r['sentence'][0], this_r['resolution_cm'][0]))
    if(len(this_r['resolution_pcs'])>0):
        print("{} >>> ICD10CM: {}".format(this_r['sentence'][0], this_r['resolution_pcs'][0]))

ICD10 codes identified for the list of patients problems found in the clinical note.
gestational diabetes mellitus >>> ICD10CM: P702
type two diabetes mellitus >>> ICD10CM: E1165
T2DM ), >>> ICD10CM: R7303
HTG-induced pancreatitis >>> ICD10CM: B252
an acute hepatitis >>> ICD10CM: B172
obesity >>> ICD10CM: E663
polyuria >>> ICD10CM: R632
polydipsia >>> ICD10CM: R631
vomiting >>> ICD10CM: R110
T2DM >>> ICD10CM: R7303
still elevated >>> ICD10CM: E7841
